In [1]:
import os
import glob
import shutil
import json
import cv2

In [11]:
FRAMES_PATH = r'D:\Projects\metal_classification\data\frames'
YOLO_DATASET_PATH = r'D:\Projects\metal_classification\data\data_for_zip'

In [12]:
id_list = ['valid_metal', 'no_metal', 'long_metal', 'thick_metal',
           'grapple', 'truck', 'magnet']
id_dict = {name: i for i, name in enumerate(id_list)}

def save_yolo_file(labels, video_name, frame_name,  save_dir):
    txt_path = os.path.join(frame_name + f'({video_name})' + '.txt')
    txt_path = os.path.join(save_dir, txt_path)
    with open(txt_path,'w') as f:
        f.write('\n'.join(labels))
        
def save_jpg_file(frame_path, video_name,  save_dir):
    fn = os.path.basename(frame_path).split('.')[0] + f'({video_name})' + '.jpg' 
    new_path = os.path.join(save_dir, fn)
    shutil.copy(frame_path,  new_path)


def create_yolo_files(frames_path, video_name, save_dir = YOLO_DATASET_PATH):
    for json_path in glob.glob(os.path.join(frames_path, '*.json')):
        with open(json_path) as f:
            data = json.load(f)
        ports_list = []
        for i in range(len(data['shapes'])):
            ports_location = data['shapes'][i]['points']
            ports_list.append(ports_location)
        
        frame_path = json_path.split('.')[0] + '.jpg'
        frame_name = os.path.basename(json_path).split('.')[0]
        save_jpg_file(frame_path, video_name, save_dir)
        labels = []
        for i in range(len(data['shapes'])):
            x_lists=[]
            y_lists=[]
            obj_ports = ports_list[i]
            for port in obj_ports:
                x_lists.append(port[0])
                y_lists.append(port[1])
            x_max = sorted(x_lists)[-1]
            x_min = sorted(x_lists)[0]
            y_max = sorted(y_lists)[-1]
            y_min = sorted(y_lists)[0]

            width = x_max-x_min
            height = y_max-y_min
            
            u,v,_ = cv2.imread(frame_path).shape
                
            center_x,center_y = round(float((x_min+width/2.0)/v),6),round(float((y_min+height/2.0)/u),6)
            
            f_width,f_height = round(float(width/v),6),round(float(height/u),6)
            
            label_id = str(id_dict[data['shapes'][i]['label']])
            
            labels.append(label_id+' '+str(center_x)+' '+str(center_y)+' '+str(f_width)+' '+str(f_height))
            
        save_yolo_file(labels,
                           video_name, frame_name, save_dir)
    print('Сonversion to yolo format completed')

In [13]:
videos_list = glob.glob(os.path.join(FRAMES_PATH, '*'))

In [14]:
videos_list

['D:\\Projects\\metal_classification\\data\\frames\\120210923000455_551',
 'D:\\Projects\\metal_classification\\data\\frames\\120210923001407_550',
 'D:\\Projects\\metal_classification\\data\\frames\\120210923083827_568']

In [15]:
for frames_path in videos_list:
    create_yolo_files(frames_path, os.path.basename(frames_path))    

Сonversion to yolo format completed
Сonversion to yolo format completed
Сonversion to yolo format completed


In [17]:
def create_train_file(data_dir, save_dir):
    train_fn = 'train.txt'
    
    labels_list = glob.glob(os.path.join(YOLO_DATASET_PATH, '*.jpg'))
    train_paths_list = []
    
#     if data_dir.split(os.path.sep)[-1]!='data':
#         raise ValueError('root directory should be \'data\'')
    
    for labels_path in labels_list:
        fn = os.path.basename(labels_path)
        jpg_fn = fn.split('.')[0] + '.jpg'
        data_path = os.path.join('data', jpg_fn)
        
        train_paths_list.append(data_path)
    
    with open(os.path.join(save_dir, train_fn), 'w') as f:
        f.write('\n'.join(train_paths_list))

In [18]:
create_train_file(data_dir = YOLO_DATASET_PATH, save_dir = YOLO_DATASET_PATH)